# NY Flight Prediction

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import bokeh as bkh
import matplotlib.pyplot as plt
from bokeh.io import output_file, show,output_notebook
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from math import pi
from bokeh.transform import cumsum
output_file("carrier_pie.html")
output_notebook()

Loading BokehJS ...

In [2]:
NYF_data=pd.read_csv('Dataset/flight_data.csv',parse_dates=[['year','month','day']])

In [3]:
NYF_data=NYF_data.rename(columns={"year_month_day": "Travel_Date"})

In [4]:
NYF_data.head()

,Travel_Date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013-01-01,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,01-01-2013 05:00
1,2013-01-01,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,01-01-2013 05:00
2,2013-01-01,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,01-01-2013 05:00
3,2013-01-01,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,01-01-2013 05:00
4,2013-01-01,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,01-01-2013 06:00


In [5]:
NYF_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 17 columns):
Travel_Date       336776 non-null datetime64[ns]
dep_time          328521 non-null float64
sched_dep_time    336776 non-null int64
dep_delay         328521 non-null float64
arr_time          328063 non-null float64
sched_arr_time    336776 non-null int64
arr_delay         327346 non-null float64
carrier           336776 non-null object
flight            336776 non-null int64
tailnum           334264 non-null object
origin            336776 non-null object
dest              336776 non-null object
air_time          327346 non-null float64
distance          336776 non-null int64
hour              336776 non-null int64
minute            336776 non-null int64
time_hour         336776 non-null object
dtypes: datetime64[ns](1), float64(5), int64(6), object(5)
memory usage: 43.7+ MB


# Departure time delay and arrival time delay missing value treatment

In [6]:
origin_carrier_grp = np.round(NYF_data.groupby(['origin','dest','carrier']).mean(),0)
origin_carrier_grp

dep_time  sched_dep_time  dep_delay  arr_time  \
origin dest carrier                                                  
EWR    ALB  EV         1627.0          1610.0       24.0    1702.0   
       ANC  UA         1635.0          1618.0       13.0    1968.0   
       ATL  9E          897.0           922.0       -6.0    1150.0   
            DL         1242.0          1231.0       12.0    1466.0   
            EV         1349.0          1335.0       22.0    1538.0   
            UA         1241.0          1212.0       16.0    1485.0   
       AUS  UA         1627.0          1603.0       15.0    1703.0   
            WN          987.0           980.0        4.0    1285.0   
       AVL  EV         1183.0          1176.0        9.0    1380.0   
       BDL  EV         1483.0          1493.0       18.0    1546.0   
            UA         1929.0          2229.0       26.0    1673.0   
       BNA  EV         1388.0          1368.0       21.0    1511.0   
            WN         1008.0          1003.0        8.0    1132.0   
       BOS  B6         1285.0          1280.0       12.0    1369.0   
            EV         1066.0          1058.0        6.0    1174.0   
            UA         1451.0          1433.0       13.0    1531.0   
       BQN  UA         2062.0          2044.0       24.0     126.0   
       BTV  EV         1339.0          1346.0       18.0    1356.0   
       BUF  EV         1133.0          1119.0       14.0    1235.0   
       BWI  EV         1567.0          1567.0       23.0    1568.0   
            WN         1242.0          1224.0       10.0    1343.0   
       BZN  UA          840.0           821.0       11.0    1113.0   
       CAE  EV         1882.0          1822.0       36.0    2064.0   
       CHS  EV         1333.0          1318.0       18.0    1502.0   
            UA         1254.0          1259.0       -5.0    1446.0   
       CLE  EV         1075.0          1084.0        8.0    1224.0   
            UA         1571.0          1554.0       12.0    1714.0   
       CLT  EV         1416.0          1406.0       23.0    1517.0   
            UA         1084.0          1106.0       -2.0    1322.0   
            US         1102.0          1106.0        3.0    1286.0   
...                       ...             ...        ...       ...   
LGA    RDU  9E         1469.0          1451.0        9.0    1650.0   
            EV         1564.0          1543.0       17.0    1742.0   
            MQ         1416.0          1418.0        8.0    1564.0   
       RIC  9E         1433.0          1402.0       17.0    1605.0   
            EV         1501.0          1518.0       17.0    1564.0   
       ROC  9E         1438.0          1516.0       -4.0    1582.0   
            EV         1328.0          1305.0       13.0    1447.0   
       RSW  9E          812.0           807.0        6.0    1133.0   
            B6         1180.0          1164.0       10.0    1495.0   
            DL         1482.0          1465.0       10.0    1717.0   
       SAV  EV          981.0           975.0        3.0    1211.0   
       SBN  EV         1952.0          1894.0       31.0    2168.0   
       SDF  9E          946.0           935.0        4.0    1186.0   
       SRQ  9E         1114.0          1091.0       15.0    1426.0   
            B6         1346.0          1328.0       10.0    1653.0   
            DL         1058.0          1045.0        6.0    1368.0   
            EV          882.0           864.0        7.0    1185.0   
       STL  AA         1404.0          1399.0        4.0    1576.0   
            EV          884.0           889.0       -4.0    1052.0   
            MQ         1347.0          1345.0       10.0    1533.0   
            WN         1219.0          1197.0       16.0    1382.0   
       SYR  9E         1961.0          2001.0       15.0    1843.0   
            EV         1511.0          1521.0       22.0    1496.0   
       TPA  B6         1183.0          1162.0       14.0    1482.0   
            DL         1347.0  

In [7]:
def dep_delay_na(org, dest, carr):
    try:
        mns= origin_carrier_grp.dep_delay[org][dest][carr]    
        indexed=NYF_data[NYF_data.origin==org][NYF_data.dest==dest][NYF_data.carrier==carr].loc[NYF_data['dep_delay'].isnull(),'dep_delay'].index
        for i in indexed:
            if np.isnan(mns):
                NYF_data.loc[i,'dep_delay']=0
            else:
                NYF_data.loc[i,'dep_delay']=mns
    except:        
        print('Combination of flight with Origin {} Destination {} and carrier {} were not found'.format(org, dest, carr))

In [8]:
def arr_delay_na(org, dest, carr):
    try:
        mns= origin_carrier_grp.arr_delay[org][dest][carr]    
        indexed=NYF_data[NYF_data.origin==org][NYF_data.dest==dest][NYF_data.carrier==carr].loc[NYF_data['arr_delay'].isnull(),'arr_delay'].index
        for i in indexed:
            if np.isnan(mns):
                NYF_data.loc[i,'arr_delay']=0
            else:
                NYF_data.loc[i,'arr_delay']=mns
    except:
        print('Combination of flight with Origin {} Destination {} and carrier {} were not found'.format(org, dest, carr))

In [9]:
NYF_data.carrier.unique()

array(['UA', 'AA', 'B6', 'DL', 'EV', 'MQ', 'US', 'WN', 'VX', 'FL', 'AS',
       '9E', 'F9', 'HA', 'YV', 'OO'], dtype=object)

In [10]:
NYF_data.origin.unique()

array(['EWR', 'LGA', 'JFK'], dtype=object)

In [11]:
NYF_data.dest.unique()

array(['IAH', 'MIA', 'BQN', 'ATL', 'ORD', 'FLL', 'IAD', 'MCO', 'PBI',
       'TPA', 'LAX', 'SFO', 'DFW', 'BOS', 'LAS', 'MSP', 'DTW', 'RSW',
       'SJU', 'PHX', 'BWI', 'CLT', 'BUF', 'DEN', 'SNA', 'MSY', 'SLC',
       'XNA', 'MKE', 'SEA', 'ROC', 'SYR', 'SRQ', 'RDU', 'CMH', 'JAX',
       'CHS', 'MEM', 'PIT', 'SAN', 'DCA', 'CLE', 'STL', 'MYR', 'JAC',
       'MDW', 'HNL', 'BNA', 'AUS', 'BTV', 'PHL', 'STT', 'EGE', 'AVL',
       'PWM', 'IND', 'SAV', 'CAK', 'HOU', 'LGB', 'DAY', 'ALB', 'BDL',
       'MHT', 'MSN', 'GSO', 'CVG', 'BUR', 'RIC', 'GSP', 'GRR', 'MCI',
       'ORF', 'SAT', 'SDF', 'PDX', 'SJC', 'OMA', 'CRW', 'OAK', 'SMF',
       'TUL', 'TYS', 'OKC', 'PVD', 'DSM', 'PSE', 'BHM', 'CAE', 'HDN',
       'BZN', 'MTJ', 'EYW', 'PSP', 'ACK', 'BGR', 'ABQ', 'ILM', 'MVY',
       'SBN', 'LEX', 'CHO', 'TVC', 'ANC', 'LGA'], dtype=object)

In [12]:
for i in NYF_data.origin.unique():
    for j in NYF_data.dest.unique():
        for k in NYF_data.carrier.unique():
            dep_delay_na(i,j,k)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Combination of flight with Origin EWR Destination IAH and carrier AA were not found
Combination of flight with Origin EWR Destination IAH and carrier B6 were not found
Combination of flight with Origin EWR Destination IAH and carrier DL were not found
Combination of flight with Origin EWR Destination IAH and carrier EV were not found
Combination of flight with Origin EWR Destination IAH and carrier MQ were not found
Combination of flight with Origin EWR Destination IAH and carrier US were not found
Combination of flight with Origin EWR Destination IAH and carrier WN were not found
Combination of flight with Origin EWR Destination IAH and carrier VX were not found
Combination of flight with Origin EWR Destination IAH and carrier FL were not found
Combination of flight with Origin EWR Destination IAH and carrier AS were not found
Combination of flight with Origin EWR Destination IAH and carrier 9E were not found
Combination of flight with Origin EWR Destination IAH and carrier F9 were no

Combination of flight with Origin EWR Destination MCO and carrier DL were not found
Combination of flight with Origin EWR Destination MCO and carrier EV were not found
Combination of flight with Origin EWR Destination MCO and carrier MQ were not found
Combination of flight with Origin EWR Destination MCO and carrier US were not found
Combination of flight with Origin EWR Destination MCO and carrier WN were not found
Combination of flight with Origin EWR Destination MCO and carrier VX were not found
Combination of flight with Origin EWR Destination MCO and carrier FL were not found
Combination of flight with Origin EWR Destination MCO and carrier AS were not found
Combination of flight with Origin EWR Destination MCO and carrier 9E were not found
Combination of flight with Origin EWR Destination MCO and carrier F9 were not found
Combination of flight with Origin EWR Destination MCO and carrier HA were not found
Combination of flight with Origin EWR Destination MCO and carrier YV were no

Combination of flight with Origin EWR Destination MSP and carrier MQ were not found
Combination of flight with Origin EWR Destination MSP and carrier US were not found
Combination of flight with Origin EWR Destination MSP and carrier WN were not found
Combination of flight with Origin EWR Destination MSP and carrier VX were not found
Combination of flight with Origin EWR Destination MSP and carrier FL were not found
Combination of flight with Origin EWR Destination MSP and carrier AS were not found
Combination of flight with Origin EWR Destination MSP and carrier F9 were not found
Combination of flight with Origin EWR Destination MSP and carrier HA were not found
Combination of flight with Origin EWR Destination MSP and carrier YV were not found
Combination of flight with Origin EWR Destination DTW and carrier AA were not found
Combination of flight with Origin EWR Destination DTW and carrier B6 were not found
Combination of flight with Origin EWR Destination DTW and carrier MQ were no

Combination of flight with Origin EWR Destination DEN and carrier AA were not found
Combination of flight with Origin EWR Destination DEN and carrier B6 were not found
Combination of flight with Origin EWR Destination DEN and carrier DL were not found
Combination of flight with Origin EWR Destination DEN and carrier EV were not found
Combination of flight with Origin EWR Destination DEN and carrier MQ were not found
Combination of flight with Origin EWR Destination DEN and carrier US were not found
Combination of flight with Origin EWR Destination DEN and carrier VX were not found
Combination of flight with Origin EWR Destination DEN and carrier FL were not found
Combination of flight with Origin EWR Destination DEN and carrier AS were not found
Combination of flight with Origin EWR Destination DEN and carrier 9E were not found
Combination of flight with Origin EWR Destination DEN and carrier F9 were not found
Combination of flight with Origin EWR Destination DEN and carrier HA were no

...

Combination of flight with Origin JFK Destination GSO and carrier FL were not found
Combination of flight with Origin JFK Destination GSO and carrier AS were not found
Combination of flight with Origin JFK Destination GSO and carrier 9E were not found
Combination of flight with Origin JFK Destination GSO and carrier F9 were not found
Combination of flight with Origin JFK Destination GSO and carrier HA were not found
Combination of flight with Origin JFK Destination GSO and carrier YV were not found
Combination of flight with Origin JFK Destination GSO and carrier OO were not found
Combination of flight with Origin JFK Destination CVG and carrier UA were not found
Combination of flight with Origin JFK Destination CVG and carrier AA were not found
Combination of flight with Origin JFK Destination CVG and carrier B6 were not found
Combination of flight with Origin JFK Destination CVG and carrier DL were not found
Combination of flight with Origin JFK Destination CVG and carrier EV were no

Combination of flight with Origin JFK Destination ORF and carrier US were not found
Combination of flight with Origin JFK Destination ORF and carrier WN were not found
Combination of flight with Origin JFK Destination ORF and carrier VX were not found
Combination of flight with Origin JFK Destination ORF and carrier FL were not found
Combination of flight with Origin JFK Destination ORF and carrier AS were not found
Combination of flight with Origin JFK Destination ORF and carrier F9 were not found
Combination of flight with Origin JFK Destination ORF and carrier HA were not found
Combination of flight with Origin JFK Destination ORF and carrier YV were not found
Combination of flight with Origin JFK Destination ORF and carrier OO were not found
Combination of flight with Origin JFK Destination SAT and carrier UA were not found
Combination of flight with Origin JFK Destination SAT and carrier AA were not found
Combination of flight with Origin JFK Destination SAT and carrier B6 were no

Combination of flight with Origin JFK Destination SMF and carrier DL were not found
Combination of flight with Origin JFK Destination SMF and carrier EV were not found
Combination of flight with Origin JFK Destination SMF and carrier MQ were not found
Combination of flight with Origin JFK Destination SMF and carrier US were not found
Combination of flight with Origin JFK Destination SMF and carrier WN were not found
Combination of flight with Origin JFK Destination SMF and carrier VX were not found
Combination of flight with Origin JFK Destination SMF and carrier FL were not found
Combination of flight with Origin JFK Destination SMF and carrier AS were not found
Combination of flight with Origin JFK Destination SMF and carrier 9E were not found
Combination of flight with Origin JFK Destination SMF and carrier F9 were not found
Combination of flight with Origin JFK Destination SMF and carrier HA were not found
Combination of flight with Origin JFK Destination SMF and carrier YV were no

Combination of flight with Origin JFK Destination CAE and carrier VX were not found
Combination of flight with Origin JFK Destination CAE and carrier FL were not found
Combination of flight with Origin JFK Destination CAE and carrier AS were not found
Combination of flight with Origin JFK Destination CAE and carrier 9E were not found
Combination of flight with Origin JFK Destination CAE and carrier F9 were not found
Combination of flight with Origin JFK Destination CAE and carrier HA were not found
Combination of flight with Origin JFK Destination CAE and carrier YV were not found
Combination of flight with Origin JFK Destination CAE and carrier OO were not found
Combination of flight with Origin JFK Destination HDN and carrier UA were not found
Combination of flight with Origin JFK Destination HDN and carrier AA were not found
Combination of flight with Origin JFK Destination HDN and carrier B6 were not found
Combination of flight with Origin JFK Destination HDN and carrier DL were no

Combination of flight with Origin JFK Destination BGR and carrier MQ were not found
Combination of flight with Origin JFK Destination BGR and carrier US were not found
Combination of flight with Origin JFK Destination BGR and carrier WN were not found
Combination of flight with Origin JFK Destination BGR and carrier VX were not found
Combination of flight with Origin JFK Destination BGR and carrier FL were not found
Combination of flight with Origin JFK Destination BGR and carrier AS were not found
Combination of flight with Origin JFK Destination BGR and carrier 9E were not found
Combination of flight with Origin JFK Destination BGR and carrier F9 were not found
Combination of flight with Origin JFK Destination BGR and carrier HA were not found
Combination of flight with Origin JFK Destination BGR and carrier YV were not found
Combination of flight with Origin JFK Destination BGR and carrier OO were not found
Combination of flight with Origin JFK Destination ABQ and carrier UA were no

In [13]:
for i in NYF_data.origin.unique():
    for j in NYF_data.dest.unique():
        for k in NYF_data.carrier.unique():
            arr_delay_na(i,j,k)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Combination of flight with Origin EWR Destination IAH and carrier AA were not found
Combination of flight with Origin EWR Destination IAH and carrier B6 were not found
Combination of flight with Origin EWR Destination IAH and carrier DL were not found
Combination of flight with Origin EWR Destination IAH and carrier EV were not found
Combination of flight with Origin EWR Destination IAH and carrier MQ were not found
Combination of flight with Origin EWR Destination IAH and carrier US were not found
Combination of flight with Origin EWR Destination IAH and carrier WN were not found
Combination of flight with Origin EWR Destination IAH and carrier VX were not found
Combination of flight with Origin EWR Destination IAH and carrier FL were not found
Combination of flight with Origin EWR Destination IAH and carrier AS were not found
Combination of flight with Origin EWR Destination IAH and carrier 9E were not found
Combination of flight with Origin EWR Destination IAH and carrier F9 were no

Combination of flight with Origin EWR Destination MCO and carrier DL were not found
Combination of flight with Origin EWR Destination MCO and carrier EV were not found
Combination of flight with Origin EWR Destination MCO and carrier MQ were not found
Combination of flight with Origin EWR Destination MCO and carrier US were not found
Combination of flight with Origin EWR Destination MCO and carrier WN were not found
Combination of flight with Origin EWR Destination MCO and carrier VX were not found
Combination of flight with Origin EWR Destination MCO and carrier FL were not found
Combination of flight with Origin EWR Destination MCO and carrier AS were not found
Combination of flight with Origin EWR Destination MCO and carrier 9E were not found
Combination of flight with Origin EWR Destination MCO and carrier F9 were not found
Combination of flight with Origin EWR Destination MCO and carrier HA were not found
Combination of flight with Origin EWR Destination MCO and carrier YV were no

Combination of flight with Origin EWR Destination MSP and carrier MQ were not found
Combination of flight with Origin EWR Destination MSP and carrier US were not found
Combination of flight with Origin EWR Destination MSP and carrier WN were not found
Combination of flight with Origin EWR Destination MSP and carrier VX were not found
Combination of flight with Origin EWR Destination MSP and carrier FL were not found
Combination of flight with Origin EWR Destination MSP and carrier AS were not found
Combination of flight with Origin EWR Destination MSP and carrier F9 were not found
Combination of flight with Origin EWR Destination MSP and carrier HA were not found
Combination of flight with Origin EWR Destination MSP and carrier YV were not found
Combination of flight with Origin EWR Destination DTW and carrier AA were not found
Combination of flight with Origin EWR Destination DTW and carrier B6 were not found
Combination of flight with Origin EWR Destination DTW and carrier MQ were no

Combination of flight with Origin EWR Destination DEN and carrier AA were not found
Combination of flight with Origin EWR Destination DEN and carrier B6 were not found
Combination of flight with Origin EWR Destination DEN and carrier DL were not found
Combination of flight with Origin EWR Destination DEN and carrier EV were not found
Combination of flight with Origin EWR Destination DEN and carrier MQ were not found
Combination of flight with Origin EWR Destination DEN and carrier US were not found
Combination of flight with Origin EWR Destination DEN and carrier VX were not found
Combination of flight with Origin EWR Destination DEN and carrier FL were not found
Combination of flight with Origin EWR Destination DEN and carrier AS were not found
Combination of flight with Origin EWR Destination DEN and carrier 9E were not found
Combination of flight with Origin EWR Destination DEN and carrier F9 were not found
Combination of flight with Origin EWR Destination DEN and carrier HA were no

...

Combination of flight with Origin JFK Destination GSO and carrier B6 were not found
Combination of flight with Origin JFK Destination GSO and carrier DL were not found
Combination of flight with Origin JFK Destination GSO and carrier EV were not found
Combination of flight with Origin JFK Destination GSO and carrier MQ were not found
Combination of flight with Origin JFK Destination GSO and carrier US were not found
Combination of flight with Origin JFK Destination GSO and carrier WN were not found
Combination of flight with Origin JFK Destination GSO and carrier VX were not found
Combination of flight with Origin JFK Destination GSO and carrier FL were not found
Combination of flight with Origin JFK Destination GSO and carrier AS were not found
Combination of flight with Origin JFK Destination GSO and carrier 9E were not found
Combination of flight with Origin JFK Destination GSO and carrier F9 were not found
Combination of flight with Origin JFK Destination GSO and carrier HA were no

Combination of flight with Origin JFK Destination MCI and carrier F9 were not found
Combination of flight with Origin JFK Destination MCI and carrier HA were not found
Combination of flight with Origin JFK Destination MCI and carrier YV were not found
Combination of flight with Origin JFK Destination MCI and carrier OO were not found
Combination of flight with Origin JFK Destination ORF and carrier UA were not found
Combination of flight with Origin JFK Destination ORF and carrier AA were not found
Combination of flight with Origin JFK Destination ORF and carrier B6 were not found
Combination of flight with Origin JFK Destination ORF and carrier DL were not found
Combination of flight with Origin JFK Destination ORF and carrier EV were not found
Combination of flight with Origin JFK Destination ORF and carrier US were not found
Combination of flight with Origin JFK Destination ORF and carrier WN were not found
Combination of flight with Origin JFK Destination ORF and carrier VX were no

Combination of flight with Origin JFK Destination OAK and carrier DL were not found
Combination of flight with Origin JFK Destination OAK and carrier EV were not found
Combination of flight with Origin JFK Destination OAK and carrier MQ were not found
Combination of flight with Origin JFK Destination OAK and carrier US were not found
Combination of flight with Origin JFK Destination OAK and carrier WN were not found
Combination of flight with Origin JFK Destination OAK and carrier VX were not found
Combination of flight with Origin JFK Destination OAK and carrier FL were not found
Combination of flight with Origin JFK Destination OAK and carrier AS were not found
Combination of flight with Origin JFK Destination OAK and carrier 9E were not found
Combination of flight with Origin JFK Destination OAK and carrier F9 were not found
Combination of flight with Origin JFK Destination OAK and carrier HA were not found
Combination of flight with Origin JFK Destination OAK and carrier YV were no

Combination of flight with Origin JFK Destination PSE and carrier DL were not found
Combination of flight with Origin JFK Destination PSE and carrier EV were not found
Combination of flight with Origin JFK Destination PSE and carrier MQ were not found
Combination of flight with Origin JFK Destination PSE and carrier US were not found
Combination of flight with Origin JFK Destination PSE and carrier WN were not found
Combination of flight with Origin JFK Destination PSE and carrier VX were not found
Combination of flight with Origin JFK Destination PSE and carrier FL were not found
Combination of flight with Origin JFK Destination PSE and carrier AS were not found
Combination of flight with Origin JFK Destination PSE and carrier 9E were not found
Combination of flight with Origin JFK Destination PSE and carrier F9 were not found
Combination of flight with Origin JFK Destination PSE and carrier HA were not found
Combination of flight with Origin JFK Destination PSE and carrier YV were no

Combination of flight with Origin JFK Destination PSP and carrier FL were not found
Combination of flight with Origin JFK Destination PSP and carrier AS were not found
Combination of flight with Origin JFK Destination PSP and carrier 9E were not found
Combination of flight with Origin JFK Destination PSP and carrier F9 were not found
Combination of flight with Origin JFK Destination PSP and carrier HA were not found
Combination of flight with Origin JFK Destination PSP and carrier YV were not found
Combination of flight with Origin JFK Destination PSP and carrier OO were not found
Combination of flight with Origin JFK Destination ACK and carrier UA were not found
Combination of flight with Origin JFK Destination ACK and carrier AA were not found
Combination of flight with Origin JFK Destination ACK and carrier DL were not found
Combination of flight with Origin JFK Destination ACK and carrier EV were not found
Combination of flight with Origin JFK Destination ACK and carrier MQ were no

In [14]:
NYF_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 17 columns):
Travel_Date       336776 non-null datetime64[ns]
dep_time          328521 non-null float64
sched_dep_time    336776 non-null int64
dep_delay         336776 non-null float64
arr_time          328063 non-null float64
sched_arr_time    336776 non-null int64
arr_delay         336776 non-null float64
carrier           336776 non-null object
flight            336776 non-null int64
tailnum           334264 non-null object
origin            336776 non-null object
dest              336776 non-null object
air_time          327346 non-null float64
distance          336776 non-null int64
hour              336776 non-null int64
minute            336776 non-null int64
time_hour         336776 non-null object
dtypes: datetime64[ns](1), float64(5), int64(6), object(5)
memory usage: 43.7+ MB


In [15]:
NYF_data.dep_delay=NYF_data.dep_delay.astype(np.int64)
NYF_data.arr_delay=NYF_data.arr_delay.astype(np.int64)

In [16]:
NYF_data.head()

,Travel_Date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013-01-01,517.0,515,2,830.0,819,11,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,01-01-2013 05:00
1,2013-01-01,533.0,529,4,850.0,830,20,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,01-01-2013 05:00
2,2013-01-01,542.0,540,2,923.0,850,33,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,01-01-2013 05:00
3,2013-01-01,544.0,545,-1,1004.0,1022,-18,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,01-01-2013 05:00
4,2013-01-01,554.0,600,-6,812.0,837,-25,DL,461,N668DN,LGA,ATL,116.0,762,6,0,01-01-2013 06:00


# Departure time and arrival time missing value treatment

In [17]:
def Hrs_Min_normalization(time):
    hr,min=np.divmod(time,100)
    if min >60:
        hr=hr+1
        min=np.mod(min,60)
    if len(str(min))==1:
        min='0'+str(min)
    return np.int64(str(hr)+str(min))

In [18]:
dep_time_nul_idx=NYF_data.loc[NYF_data.dep_time.isnull(),'dep_time'].index

In [19]:
for i in dep_time_nul_idx:   
    NYF_data.loc[i,'dep_time']=Hrs_Min_normalization(NYF_data.loc[i,'sched_dep_time']+NYF_data.loc[i,'dep_delay'])

In [20]:
arr_time_nul_idx=NYF_data.loc[NYF_data.arr_time.isnull(),'arr_time'].index

In [21]:
for i in arr_time_nul_idx:   
    NYF_data.loc[i,'arr_time']=Hrs_Min_normalization(NYF_data.loc[i,'sched_arr_time']+NYF_data.loc[i,'arr_delay'])

In [22]:
NYF_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 17 columns):
Travel_Date       336776 non-null datetime64[ns]
dep_time          336776 non-null float64
sched_dep_time    336776 non-null int64
dep_delay         336776 non-null int64
arr_time          336776 non-null float64
sched_arr_time    336776 non-null int64
arr_delay         336776 non-null int64
carrier           336776 non-null object
flight            336776 non-null int64
tailnum           334264 non-null object
origin            336776 non-null object
dest              336776 non-null object
air_time          327346 non-null float64
distance          336776 non-null int64
hour              336776 non-null int64
minute            336776 non-null int64
time_hour         336776 non-null object
dtypes: datetime64[ns](1), float64(3), int64(8), object(5)
memory usage: 43.7+ MB


In [23]:
NYF_data[NYF_data.air_time.isnull()].head()

,Travel_Date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
471,2013-01-01,1525.0,1530,-5,1934.0,1805,7,MQ,4525,N719MQ,LGA,XNA,NaN,1147,15,30,01-01-2013 15:00
477,2013-01-01,1528.0,1459,29,2002.0,1647,17,EV,3806,N17108,EWR,STL,NaN,872,14,59,01-01-2013 14:00
615,2013-01-01,1740.0,1745,-5,2158.0,2020,7,MQ,4413,N739MQ,LGA,XNA,NaN,1147,17,45,01-01-2013 17:00
643,2013-01-01,1807.0,1738,29,2251.0,2103,3,UA,1228,N31412,EWR,SAN,NaN,2425,17,38,01-01-2013 17:00
725,2013-01-01,1939.0,1840,59,29.0,2151,11,9E,3325,N905XJ,JFK,DFW,NaN,1391,18,40,01-01-2013 18:00


In [24]:
NYF_data.loc[NYF_data['air_time'].isnull(),'arr_time'].head()

471    1934.0
477    2002.0
615    2158.0
643    2251.0
725      29.0
Name: arr_time, dtype: float64

In [25]:
NYF_data.dep_time=NYF_data.dep_time.astype(np.int64)
NYF_data.arr_time=NYF_data.arr_time.astype(np.int64)

In [26]:
NYF_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 17 columns):
Travel_Date       336776 non-null datetime64[ns]
dep_time          336776 non-null int64
sched_dep_time    336776 non-null int64
dep_delay         336776 non-null int64
arr_time          336776 non-null int64
sched_arr_time    336776 non-null int64
arr_delay         336776 non-null int64
carrier           336776 non-null object
flight            336776 non-null int64
tailnum           334264 non-null object
origin            336776 non-null object
dest              336776 non-null object
air_time          327346 non-null float64
distance          336776 non-null int64
hour              336776 non-null int64
minute            336776 non-null int64
time_hour         336776 non-null object
dtypes: datetime64[ns](1), float64(1), int64(10), object(5)
memory usage: 43.7+ MB


In [27]:
NYF_data.head()

,Travel_Date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013-01-01,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,01-01-2013 05:00
1,2013-01-01,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,01-01-2013 05:00
2,2013-01-01,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,01-01-2013 05:00
3,2013-01-01,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,01-01-2013 05:00
4,2013-01-01,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116.0,762,6,0,01-01-2013 06:00


# On air time duration missing value treatment

In [28]:
origin_carrier_grp.air_time['EWR']['ALB']

carrier
EV    32.0
Name: air_time, dtype: float64

In [29]:
def air_time_na(org, dest, carr):
    try:
        mns= origin_carrier_grp.air_time[org][dest][carr]    
        indexed=NYF_data[NYF_data.origin==org][NYF_data.dest==dest][NYF_data.carrier==carr].loc[NYF_data['air_time'].isnull(),'air_time'].index
        for i in indexed:
            if np.isnan(mns):
                NYF_data.loc[i,'air_time']=NYF_data.arr_time.mean()
            else:
                NYF_data.loc[i,'air_time']=mns
    except:        
        print('Air time for combination of flight with Origin {} Destination {} and carrier {} were not found'.format(org, dest, carr))

In [30]:
NYF_data[NYF_data['air_time'].isnull()]

,Travel_Date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
471,2013-01-01,1525,1530,-5,1934,1805,7,MQ,4525,N719MQ,LGA,XNA,NaN,1147,15,30,01-01-2013 15:00
477,2013-01-01,1528,1459,29,2002,1647,17,EV,3806,N17108,EWR,STL,NaN,872,14,59,01-01-2013 14:00
615,2013-01-01,1740,1745,-5,2158,2020,7,MQ,4413,N739MQ,LGA,XNA,NaN,1147,17,45,01-01-2013 17:00
643,2013-01-01,1807,1738,29,2251,2103,3,UA,1228,N31412,EWR,SAN,NaN,2425,17,38,01-01-2013 17:00
725,2013-01-01,1939,1840,59,29,2151,11,9E,3325,N905XJ,JFK,DFW,NaN,1391,18,40,01-01-2013 18:00
733,2013-01-01,1952,1930,22,2358,2207,34,EV,4333,N11194,EWR,TUL,NaN,1215,19,30,01-01-2013 19:00
754,2013-01-01,2016,1930,46,2251,2220,31,EV,4204,N14168,EWR,OKC,NaN,1325,19,30,01-01-2013 19:00
838,2013-01-01,1646,1630,16,1828,1815,13,EV,4308,N18120,EWR,RDU,NaN,416,16,30,01-01-2013 16:00
839,2013-01-01,1941,1935,6,2239,2240,-1,AA,791,N3EHAA,LGA,DFW,NaN,1389,19,35,01-01-2013 19:00
840,2013-01-01,1506,1500,6,1823,1825,-2,AA,1925,N3EVAA,LGA,MIA,NaN,1096,15,0,01-01-2013 15:00


In [31]:
for i in NYF_data.origin.unique():
    for j in NYF_data.dest.unique():
        for k in NYF_data.carrier.unique():
            air_time_na(i,j,k)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Air time for combination of flight with Origin EWR Destination IAH and carrier AA were not found
Air time for combination of flight with Origin EWR Destination IAH and carrier B6 were not found
Air time for combination of flight with Origin EWR Destination IAH and carrier DL were not found
Air time for combination of flight with Origin EWR Destination IAH and carrier EV were not found
Air time for combination of flight with Origin EWR Destination IAH and carrier MQ were not found
Air time for combination of flight with Origin EWR Destination IAH and carrier US were not found
Air time for combination of flight with Origin EWR Destination IAH and carrier WN were not found
Air time for combination of flight with Origin EWR Destination IAH and carrier VX were not found
Air time for combination of flight with Origin EWR Destination IAH and carrier FL were not found
Air time for combination of flight with Origin EWR Destination IAH and carrier AS were not found
Air time for combination of fl

Air time for combination of flight with Origin EWR Destination IAD and carrier MQ were not found
Air time for combination of flight with Origin EWR Destination IAD and carrier US were not found
Air time for combination of flight with Origin EWR Destination IAD and carrier WN were not found
Air time for combination of flight with Origin EWR Destination IAD and carrier VX were not found
Air time for combination of flight with Origin EWR Destination IAD and carrier FL were not found
Air time for combination of flight with Origin EWR Destination IAD and carrier AS were not found
Air time for combination of flight with Origin EWR Destination IAD and carrier 9E were not found
Air time for combination of flight with Origin EWR Destination IAD and carrier F9 were not found
Air time for combination of flight with Origin EWR Destination IAD and carrier HA were not found
Air time for combination of flight with Origin EWR Destination IAD and carrier YV were not found
Air time for combination of fl

Air time for combination of flight with Origin EWR Destination BOS and carrier AA were not found
Air time for combination of flight with Origin EWR Destination BOS and carrier DL were not found
Air time for combination of flight with Origin EWR Destination BOS and carrier MQ were not found
Air time for combination of flight with Origin EWR Destination BOS and carrier US were not found
Air time for combination of flight with Origin EWR Destination BOS and carrier WN were not found
Air time for combination of flight with Origin EWR Destination BOS and carrier VX were not found
Air time for combination of flight with Origin EWR Destination BOS and carrier FL were not found
Air time for combination of flight with Origin EWR Destination BOS and carrier AS were not found
Air time for combination of flight with Origin EWR Destination BOS and carrier 9E were not found
Air time for combination of flight with Origin EWR Destination BOS and carrier F9 were not found
Air time for combination of fl

Air time for combination of flight with Origin EWR Destination BWI and carrier MQ were not found
Air time for combination of flight with Origin EWR Destination BWI and carrier US were not found
Air time for combination of flight with Origin EWR Destination BWI and carrier VX were not found
Air time for combination of flight with Origin EWR Destination BWI and carrier FL were not found
Air time for combination of flight with Origin EWR Destination BWI and carrier AS were not found
Air time for combination of flight with Origin EWR Destination BWI and carrier 9E were not found
Air time for combination of flight with Origin EWR Destination BWI and carrier F9 were not found
Air time for combination of flight with Origin EWR Destination BWI and carrier HA were not found
Air time for combination of flight with Origin EWR Destination BWI and carrier YV were not found
Air time for combination of flight with Origin EWR Destination BWI and carrier OO were not found
Air time for combination of fl

...

Air time for combination of flight with Origin JFK Destination MCI and carrier F9 were not found
Air time for combination of flight with Origin JFK Destination MCI and carrier HA were not found
Air time for combination of flight with Origin JFK Destination MCI and carrier YV were not found
Air time for combination of flight with Origin JFK Destination MCI and carrier OO were not found
Air time for combination of flight with Origin JFK Destination ORF and carrier UA were not found
Air time for combination of flight with Origin JFK Destination ORF and carrier AA were not found
Air time for combination of flight with Origin JFK Destination ORF and carrier B6 were not found
Air time for combination of flight with Origin JFK Destination ORF and carrier DL were not found
Air time for combination of flight with Origin JFK Destination ORF and carrier EV were not found
Air time for combination of flight with Origin JFK Destination ORF and carrier US were not found
Air time for combination of fl

Air time for combination of flight with Origin JFK Destination CRW and carrier UA were not found
Air time for combination of flight with Origin JFK Destination CRW and carrier AA were not found
Air time for combination of flight with Origin JFK Destination CRW and carrier B6 were not found
Air time for combination of flight with Origin JFK Destination CRW and carrier DL were not found
Air time for combination of flight with Origin JFK Destination CRW and carrier EV were not found
Air time for combination of flight with Origin JFK Destination CRW and carrier MQ were not found
Air time for combination of flight with Origin JFK Destination CRW and carrier US were not found
Air time for combination of flight with Origin JFK Destination CRW and carrier WN were not found
Air time for combination of flight with Origin JFK Destination CRW and carrier VX were not found
Air time for combination of flight with Origin JFK Destination CRW and carrier FL were not found
Air time for combination of fl

Air time for combination of flight with Origin JFK Destination PSE and carrier DL were not found
Air time for combination of flight with Origin JFK Destination PSE and carrier EV were not found
Air time for combination of flight with Origin JFK Destination PSE and carrier MQ were not found
Air time for combination of flight with Origin JFK Destination PSE and carrier US were not found
Air time for combination of flight with Origin JFK Destination PSE and carrier WN were not found
Air time for combination of flight with Origin JFK Destination PSE and carrier VX were not found
Air time for combination of flight with Origin JFK Destination PSE and carrier FL were not found
Air time for combination of flight with Origin JFK Destination PSE and carrier AS were not found
Air time for combination of flight with Origin JFK Destination PSE and carrier 9E were not found
Air time for combination of flight with Origin JFK Destination PSE and carrier F9 were not found
Air time for combination of fl

Air time for combination of flight with Origin JFK Destination PSP and carrier UA were not found
Air time for combination of flight with Origin JFK Destination PSP and carrier AA were not found
Air time for combination of flight with Origin JFK Destination PSP and carrier B6 were not found
Air time for combination of flight with Origin JFK Destination PSP and carrier DL were not found
Air time for combination of flight with Origin JFK Destination PSP and carrier EV were not found
Air time for combination of flight with Origin JFK Destination PSP and carrier MQ were not found
Air time for combination of flight with Origin JFK Destination PSP and carrier US were not found
Air time for combination of flight with Origin JFK Destination PSP and carrier WN were not found
Air time for combination of flight with Origin JFK Destination PSP and carrier FL were not found
Air time for combination of flight with Origin JFK Destination PSP and carrier AS were not found
Air time for combination of fl

Air time for combination of flight with Origin JFK Destination SBN and carrier UA were not found
Air time for combination of flight with Origin JFK Destination SBN and carrier AA were not found
Air time for combination of flight with Origin JFK Destination SBN and carrier B6 were not found
Air time for combination of flight with Origin JFK Destination SBN and carrier DL were not found
Air time for combination of flight with Origin JFK Destination SBN and carrier EV were not found
Air time for combination of flight with Origin JFK Destination SBN and carrier MQ were not found
Air time for combination of flight with Origin JFK Destination SBN and carrier US were not found
Air time for combination of flight with Origin JFK Destination SBN and carrier WN were not found
Air time for combination of flight with Origin JFK Destination SBN and carrier VX were not found
Air time for combination of flight with Origin JFK Destination SBN and carrier FL were not found
Air time for combination of fl

In [32]:
NYF_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 17 columns):
Travel_Date       336776 non-null datetime64[ns]
dep_time          336776 non-null int64
sched_dep_time    336776 non-null int64
dep_delay         336776 non-null int64
arr_time          336776 non-null int64
sched_arr_time    336776 non-null int64
arr_delay         336776 non-null int64
carrier           336776 non-null object
flight            336776 non-null int64
tailnum           334264 non-null object
origin            336776 non-null object
dest              336776 non-null object
air_time          336776 non-null float64
distance          336776 non-null int64
hour              336776 non-null int64
minute            336776 non-null int64
time_hour         336776 non-null object
dtypes: datetime64[ns](1), float64(1), int64(10), object(5)
memory usage: 43.7+ MB


In [33]:
NYF_data=NYF_data.drop(columns='tailnum')

In [34]:
NYF_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 16 columns):
Travel_Date       336776 non-null datetime64[ns]
dep_time          336776 non-null int64
sched_dep_time    336776 non-null int64
dep_delay         336776 non-null int64
arr_time          336776 non-null int64
sched_arr_time    336776 non-null int64
arr_delay         336776 non-null int64
carrier           336776 non-null object
flight            336776 non-null int64
origin            336776 non-null object
dest              336776 non-null object
air_time          336776 non-null float64
distance          336776 non-null int64
hour              336776 non-null int64
minute            336776 non-null int64
time_hour         336776 non-null object
dtypes: datetime64[ns](1), float64(1), int64(10), object(4)
memory usage: 41.1+ MB


In [35]:
NYF_data['Month']=NYF_data.Travel_Date.dt.month
NYF_data['Year']=NYF_data.Travel_Date.dt.year
NYF_data['DayofWeek']=NYF_data.Travel_Date.dt.weekday_name

In [65]:
NYF_data.to_csv('Dataset/NYF_data_no_NA.csv',index=False)